# Pythonでブロックチェーンを操作する

[PythonでSymbolブロックチェーンの送金プログラムを書いてみる。](https://qiita.com/nem_takanobu/items/f3c02caa17ad385b6155)

In [1]:
# unhexlifyを使用して秘密鍵をバイナリ化
from binascii import unhexlify
b = unhexlify("ひみつ")

In [2]:
# PrivateKeyクラスを生成
from symbolchain.core.CryptoTypes import PrivateKey
prikey = PrivateKey(b)

In [3]:
# KeyPairを作成
from symbolchain.core.sym.KeyPair import KeyPair
keypair = KeyPair(prikey)

In [4]:
# PublicKeyを作成
pubkey = keypair.public_key
print(str(keypair.public_key))

68641FB644954A5AD08F03B17CD028CCA01558DC610E3EF7ABE0ABF63E4236E8


In [5]:
# Symbolのファサードを生成
from symbolchain.core.facade.SymFacade import SymFacade
facade = SymFacade('public_test')

In [6]:
# アドレスを生成
address = facade.network.public_key_to_address(pubkey)
print(str(address))

TDIFOJ4FFN7F3OF6IRM5IMQ7WCTTFD7BDXDY7FQ


In [7]:
# トランザクションの有効期限を設定
import datetime
deadline = (int((datetime.datetime.today() + datetime.timedelta(hours=2)).timestamp()) - 1616694977) * 1000

In [8]:
# トランザクション作成
tx = facade.transaction_factory.create({
  'type': 'transfer',
  'signer_public_key': pubkey,
  'fee': 100000,
  'deadline': deadline,
  'recipient_address': address,
  'mosaics': [(0x091F837E059AE13C, 1000000)],
  'message': bytes(1)
})

In [9]:
# 署名
signature = facade.sign_transaction(keypair, tx)
tx.signature = signature.bytes

In [10]:
# JSON化
from binascii import hexlify
payload = {"payload": hexlify(tx.serialize()).decode('utf8').upper()}

import json
json = json.dumps(payload)

In [11]:
# アナウンス
headers = {'Content-type': 'application/json'}
import http.client

conn = http.client.HTTPConnection("sym-test-01.opening-line.jp",3000)
conn.request("PUT", "/transactions", json,headers)

response = conn.getresponse()
print(response.status, response.reason)

202 Accepted


In [12]:
# 確認
hash = facade.hash_transaction(tx)
print('https://sym-test-01.opening-line.jp:3001/transactionStatus/' + str(hash))

https://sym-test-01.opening-line.jp:3001/transactionStatus/433691BC813BD37E5239B24375CF33806F1A88DC631CB3B761DFDD9934EA851F
